`damcard_1905_ダムカード配布所一覧` からDBに投入可能なデータにするための前処理

https://drive.google.com/file/d/1MUkJJ7C3xYGPVKuT8hEHxJnb2m_9iwQq/view?usp=sharing

In [84]:
import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 1000
XLS_FILE_PATH = './damcard_1905_ダムカード配布所一覧.xlsx'
df_dam_card_places = pd.read_excel(XLS_FILE_PATH, skiprows=0)

In [85]:
df_dam_card_places.head(5)

,番号,水系名,河川名,ダム名,ver,配布場所,配布日時,ダム所在県名,配布場所の住所,ホームページURL
0,NaN,石狩川,石狩川,大雪ダム,1.2,大雪ダム管理支所,9:00～17:00\n（土・日・祝日を含む）,北海道,上川郡上川町字層雲峡大学平,http://www.hkd.mlit.go.jp/as/tisui/ho928l00000...
1,NaN,石狩川,忠別川,忠別ダム,2.0,忠別ダム管理支所,8:30～17:15\n（土・日・祝日を含む）,NaN,上川郡東川町ノカナン,http://www.hkd.mlit.go.jp/as/tisui/ho928l00000...
2,NaN,石狩川,空知川,金山ダム,2.1,金山ダム管理支所,9:00～17:00\n（土・日・祝日を含む）,NaN,空知郡南富良野町字金山,http://www.hkd.mlit.go.jp/sp/sorati_kasen/kluh...
3,NaN,石狩川,空知川,滝里ダム,1.2,滝里ダム管理支所,9:00～17:00\n（土・日・祝日を含む）,NaN,芦別市滝里町６８３,http://www.hkd.mlit.go.jp/sp/sorati_kasen/kluh...
4,NaN,石狩川,幾春別川,桂沢ダム,2.1,幾春別川ダム建設事業所,9:00～17:00\n(平日のみ),NaN,三笠市幾春別山手町91-1,http://www.hkd.mlit.go.jp/sp/iwamizawa_kasen/k...


データのクレンジング

In [86]:
def purefy(item):
    if item == np.nan or type(item) == float:
        return item
    item = item.replace('\n', ' ').replace('\u3000', ' ').replace('※', '').replace('（', '(').replace('）',')')
    return re.sub('\s+', ' ', item).strip()

In [87]:
df_dam_card_places = df_dam_card_places.applymap(purefy)

水系名、河川名、ダム名、verカラムを直前のvalidな値でfillする

In [88]:
df_dam_card_places['水系名'] = df_dam_card_places['水系名'].fillna(method='ffill')
df_dam_card_places['河川名'] = df_dam_card_places['河川名'].fillna(method='ffill')
df_dam_card_places['ダム名'] = df_dam_card_places['ダム名'].fillna(method='ffill')
df_dam_card_places['ver'] = df_dam_card_places['ver'].fillna(method='ffill')
df_dam_card_places['ダム所在県名'] = df_dam_card_places['ダム所在県名'].fillna(method='ffill')
df_dam_card_places['配布場所'] = df_dam_card_places['配布場所'].fillna(method='ffill')

### 配布日時カラムのクレンジング

In [89]:
import mojimoji #https://pypi.org/project/mojimoji/0.0.9/
def zen_to_han(item):
    if item == np.nan or type(item) == float:
        return item
    return mojimoji.zen_to_han(item, kana=False) #カナ以外は半角に

def jikan_and_fun(item):
    if item == np.nan or type(item) == float:
        return item
    hours = [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
    minutes = ['00', '15', '30', '45']
    for hour in hours:
        item = item.replace('{}時'.format(hour), '{}:'.format(hour))
    for minute in minutes:
        item = item.replace('{}分'.format(minute), '{}'.format(minute))
            
    return item

In [90]:
df_dam_card_places['配布日時'] = df_dam_card_places['配布日時'].apply(zen_to_han).apply(jikan_and_fun)


ダム名にダムの状態を表す記述がある。例`(建設中)` これらを除き、dam_statusカラムを作る

In [91]:
def split_by_parenthesis(item):
    w = re.search(r"\((.*?)\)", item)
    if w:
        return w.group()
    return w

In [92]:
df_dam_card_places['dam_status'] = df_dam_card_places['ダム名'].apply(split_by_parenthesis)

ダム名からVlookupに使える名前を抽出し、新たに「dam_name」カラムを作る

In [93]:
def generate_dam_name(item):
    if type(item) == float:
        return ''
    return re.sub(r"\((.*?)\)" ,'' , item.replace('ダム', ''))

In [94]:
df_dam_card_places['dam_name']=df_dam_card_places['ダム名'].apply(generate_dam_name)

In [95]:
df_dam_card_places.head(5)

,番号,水系名,河川名,ダム名,ver,配布場所,配布日時,ダム所在県名,配布場所の住所,ホームページURL,dam_status,dam_name
0,NaN,石狩川,石狩川,大雪ダム,1.2,大雪ダム管理支所,9:00~17:00 (土・日・祝日を含む),北海道,上川郡上川町字層雲峡大学平,http://www.hkd.mlit.go.jp/as/tisui/ho928l00000...,None,大雪
1,NaN,石狩川,忠別川,忠別ダム,2.0,忠別ダム管理支所,8:30~17:15 (土・日・祝日を含む),北海道,上川郡東川町ノカナン,http://www.hkd.mlit.go.jp/as/tisui/ho928l00000...,None,忠別
2,NaN,石狩川,空知川,金山ダム,2.1,金山ダム管理支所,9:00~17:00 (土・日・祝日を含む),北海道,空知郡南富良野町字金山,http://www.hkd.mlit.go.jp/sp/sorati_kasen/kluh...,None,金山
3,NaN,石狩川,空知川,滝里ダム,1.2,滝里ダム管理支所,9:00~17:00 (土・日・祝日を含む),北海道,芦別市滝里町６８３,http://www.hkd.mlit.go.jp/sp/sorati_kasen/kluh...,None,滝里
4,NaN,石狩川,幾春別川,桂沢ダム,2.1,幾春別川ダム建設事業所,9:00~17:00 (平日のみ),北海道,三笠市幾春別山手町91-1,http://www.hkd.mlit.go.jp/sp/iwamizawa_kasen/k...,None,桂沢


------

In [96]:
CSV_FILE_PATH = './Dam-Data.csv'
df_dam = pd.read_csv(CSV_FILE_PATH, skiprows=0)

### ダムcsvファイルで県名コラムを作成する

In [97]:
def extract_prefecture(item):
    return re.search(r"^(.{2}[都道府県]|.{3}県)", item).group()

In [98]:
df_dam['prefecture']=df_dam['W01_013'].apply(extract_prefecture)

Vlookupするためのマッピング作成

In [99]:
prefs = list(df_dam['prefecture'])
keys = list(df_dam['W01_001'])
values = list(df_dam['W01_002'])


In [100]:
mapping = {}   #{'駒込': 191}
for k,v,p in zip(keys, values, prefs):
    mapping['{}-{}'.format(p,k)]=v

In [101]:
dam_ids = []
dam_names = list(df_dam_card_places['dam_name'])
dam_location_prefs = list(df_dam_card_places['ダム所在県名'])
for name, pref in zip(dam_names, dam_location_prefs):
    lookup_key = '{}-{}'.format(pref,name)
    dam_ids.append(mapping.get(lookup_key, None))

In [102]:
df_dam_card_places['dam_id']=dam_ids
df_dam_card_places['dam_id'] = df_dam_card_places['dam_id']

### 配布場所でグルーピングしてplace_idを付与

In [103]:
df_dam_card_places['card_distribution_place_id']=df_dam_card_places.groupby(['配布場所']).ngroup()

### 最後にdamの番号カラムにrecord_idを加える

In [104]:
df_dam_card_places['番号']=np.arange(len(df_dam_card_places))

In [105]:
df_dam_card_places.set_index('番号', inplace=True)

### csv出力

In [106]:
df_dam_card_places.to_csv('./df_dam_card_places.csv')

---------------

In [107]:
target_columns = ['card_distribution_place_id', 'dam_id', 'dam_name', 'ホームページURL', '配布場所の住所', '配布日時', '配布場所', 'ダム所在県名']
df_dam_card_places_set = df_dam_card_places[target_columns].copy()

In [108]:
df_dam_card_places_set.drop_duplicates(subset='card_distribution_place_id', keep='first', inplace=True)

In [109]:
df_dam_card_places_set.to_csv('./dam_card_places_set.csv')